# Rekurentne neuronske mreže

U prethodnom modulu koristili smo bogate semantičke reprezentacije teksta i jednostavan linearni klasifikator na vrhu ugrađenih vektora. Ova arhitektura hvata agregirano značenje riječi u rečenici, ali ne uzima u obzir **redoslijed** riječi, jer operacija agregacije na vrhu ugrađenih vektora uklanja tu informaciju iz izvornog teksta. Budući da ti modeli ne mogu modelirati redoslijed riječi, nisu sposobni riješiti složenije ili dvosmislene zadatke poput generiranja teksta ili odgovaranja na pitanja.

Kako bismo uhvatili značenje sekvenci teksta, trebamo koristiti drugu arhitekturu neuronske mreže, koja se naziva **rekurentna neuronska mreža** ili RNN. U RNN-u, rečenicu prosljeđujemo kroz mrežu jedan simbol po simbol, a mreža proizvodi određeno **stanje**, koje zatim ponovno prosljeđujemo mreži zajedno sa sljedećim simbolom.

S obzirom na ulaznu sekvencu tokena $X_0,\dots,X_n$, RNN stvara sekvencu blokova neuronske mreže i trenira ovu sekvencu od početka do kraja koristeći povratnu propagaciju. Svaki blok mreže uzima par $(X_i,S_i)$ kao ulaz i proizvodi $S_{i+1}$ kao rezultat. Konačno stanje $S_n$ ili izlaz $X_n$ ide u linearni klasifikator kako bi se proizveo rezultat. Svi blokovi mreže dijele iste težine i treniraju se od početka do kraja koristeći jednu povratnu propagaciju.

Budući da se vektori stanja $S_0,\dots,S_n$ prosljeđuju kroz mrežu, ona može naučiti sekvencijalne ovisnosti između riječi. Na primjer, kada se riječ *ne* pojavi negdje u sekvenci, mreža može naučiti negirati određene elemente unutar vektora stanja, što rezultira negacijom.

> Budući da svi blokovi RNN-a na slici dijele iste težine, ista slika može se prikazati kao jedan blok (desno) s povratnom petljom, koja prosljeđuje izlazno stanje mreže natrag na ulaz.

Pogledajmo kako rekurentne neuronske mreže mogu pomoći u klasifikaciji našeg skupa podataka o vijestima.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Jednostavni RNN klasifikator

Kod jednostavnog RNN-a, svaka rekurentna jedinica je jednostavna linearna mreža koja uzima spojeni ulazni vektor i vektor stanja te proizvodi novi vektor stanja. PyTorch predstavlja ovu jedinicu s klasom `RNNCell`, a mreže takvih jedinica - kao sloj `RNN`.

Za definiranje RNN klasifikatora, prvo ćemo primijeniti sloj za ugrađivanje kako bismo smanjili dimenzionalnost ulaznog vokabulara, a zatim dodati RNN sloj na vrh:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Napomena:** Ovdje koristimo neuvježbani sloj ugradnje radi jednostavnosti, ali za još bolje rezultate možemo koristiti unaprijed uvježbani sloj ugradnje s Word2Vec ili GloVe ugradnjama, kako je opisano u prethodnoj jedinici. Za bolje razumijevanje, možda biste željeli prilagoditi ovaj kod kako bi radio s unaprijed uvježbanim ugradnjama.

U našem slučaju koristit ćemo učitavač podataka s popunjavanjem (padded data loader), tako da će svaki batch sadržavati određeni broj sekvenci iste duljine. RNN sloj će primiti sekvencu tenzora ugradnje i proizvesti dva izlaza:
* $x$ je sekvenca izlaza RNN ćelija na svakom koraku
* $h$ je konačno skriveno stanje za zadnji element sekvence

Zatim primjenjujemo potpuno povezani linearni klasifikator kako bismo dobili broj klase.

> **Napomena:** RNN-ove je prilično teško trenirati jer, kada se RNN ćelije razviju duž duljine sekvence, broj slojeva uključenih u povratnu propagaciju postaje prilično velik. Stoga trebamo odabrati malu stopu učenja i trenirati mrežu na većem skupu podataka kako bismo postigli dobre rezultate. To može potrajati dosta dugo, pa je preporučljivo koristiti GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Long Short Term Memory (LSTM)

Jedan od glavnih problema klasičnih RNN-a je takozvani problem **nestajućih gradijenata**. Budući da se RNN-i treniraju od kraja do početka u jednom prolazu unatrag, teško prenose pogrešku do prvih slojeva mreže, zbog čega mreža ne može naučiti odnose između udaljenih tokena. Jedan od načina za izbjegavanje ovog problema je uvođenje **eksplicitnog upravljanja stanjem** pomoću takozvanih **vrata**. Dvije najpoznatije arhitekture ovog tipa su: **Long Short Term Memory** (LSTM) i **Gated Relay Unit** (GRU).

![Slika koja prikazuje primjer ćelije Long Short Term Memory](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM mreža organizirana je na način sličan RNN-u, ali postoje dva stanja koja se prenose iz sloja u sloj: stvarno stanje $c$ i skriveni vektor $h$. U svakoj jedinici, skriveni vektor $h_i$ se spaja s ulazom $x_i$, i oni kontroliraju što se događa sa stanjem $c$ putem **vrata**. Svaka vrata su neuronska mreža sa sigmoidnom aktivacijom (izlaz u rasponu $[0,1]$), koja se može smatrati bitmaskom kada se pomnoži s vektorom stanja. Postoje sljedeća vrata (s lijeva na desno na slici iznad):
* **vrata zaborava** uzimaju skriveni vektor i određuju koje komponente vektora $c$ trebamo zaboraviti, a koje proslijediti dalje.
* **ulazna vrata** uzimaju neke informacije iz ulaza i skrivenog vektora te ih ubacuju u stanje.
* **izlazna vrata** transformiraju stanje putem nekog linearnog sloja s $\tanh$ aktivacijom, zatim odabiru neke od njegovih komponenti koristeći skriveni vektor $h_i$ kako bi proizveli novo stanje $c_{i+1}$.

Komponente stanja $c$ mogu se smatrati nekim zastavicama koje se mogu uključiti ili isključiti. Na primjer, kada u sekvenci naiđemo na ime *Alice*, možda ćemo pretpostaviti da se odnosi na ženski lik i podići zastavicu u stanju koja označava da imamo ženski imenicu u rečenici. Kada kasnije naiđemo na frazu *and Tom*, podići ćemo zastavicu koja označava da imamo množinsku imenicu. Tako manipulacijom stanja možemo navodno pratiti gramatička svojstva dijelova rečenice.

> **Note**: Odličan resurs za razumijevanje unutarnje strukture LSTM-a je ovaj sjajan članak [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) Christophera Olaha.

Iako unutarnja struktura LSTM ćelije može izgledati složeno, PyTorch skriva ovu implementaciju unutar klase `LSTMCell` i pruža objekt `LSTM` za predstavljanje cijelog LSTM sloja. Stoga će implementacija LSTM klasifikatora biti prilično slična jednostavnom RNN-u koji smo vidjeli ranije:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Pakirane sekvence

U našem primjeru morali smo popuniti sve sekvence u minibatchu nulama. Iako to rezultira određenim gubitkom memorije, kod RNN-a je još kritičnije što se dodatne RNN ćelije stvaraju za popunjene ulazne stavke, koje sudjeluju u treningu, ali ne nose nikakve važne ulazne informacije. Bilo bi puno bolje trenirati RNN samo do stvarne duljine sekvence.

Kako bismo to postigli, u PyTorchu je uveden poseban format za pohranu popunjenih sekvenci. Pretpostavimo da imamo ulazni popunjeni minibatch koji izgleda ovako:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Ovdje 0 predstavlja popunjene vrijednosti, a stvarni vektor duljina ulaznih sekvenci je `[5,3,1]`.

Kako bismo učinkovito trenirali RNN s popunjenim sekvencama, želimo započeti trening prve grupe RNN ćelija s velikim minibatchom (`[1,6,9]`), ali zatim završiti obradu treće sekvence i nastaviti trening s kraćim minibatchevima (`[2,7]`, `[3,8]`), i tako dalje. Dakle, pakirana sekvenca je predstavljena kao jedan vektor - u našem slučaju `[1,6,9,2,7,3,8,4,5]`, i vektor duljina (`[5,3,1]`), iz kojeg lako možemo rekonstruirati originalni popunjeni minibatch.

Za stvaranje pakirane sekvence možemo koristiti funkciju `torch.nn.utils.rnn.pack_padded_sequence`. Sve rekurzivne slojeve, uključujući RNN, LSTM i GRU, podržavaju pakirane sekvence kao ulaz i proizvode pakirani izlaz, koji se može dekodirati pomoću `torch.nn.utils.rnn.pad_packed_sequence`.

Kako bismo mogli proizvesti pakiranu sekvencu, trebamo proslijediti vektor duljina mreži, i stoga nam je potrebna drugačija funkcija za pripremu minibatcheva:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Stvarna mreža bila bi vrlo slična `LSTMClassifier` gore, ali `forward` prolaz će primiti i podskup s podacima (padded minibatch) i vektor duljina sekvenci. Nakon izračuna ugradnje (embedding), izračunavamo zapakiranu sekvencu, prosljeđujemo je LSTM sloju, a zatim rezultat ponovno raspakiramo.

> **Napomena**: Zapravo ne koristimo raspakirani rezultat `x`, jer koristimo izlaz iz skrivenih slojeva u sljedećim izračunima. Stoga možemo u potpunosti ukloniti raspakiranje iz ovog koda. Razlog zašto ga ovdje uključujemo je taj da vam omogućimo jednostavno modificiranje ovog koda, u slučaju da trebate koristiti izlaz mreže u daljnjim izračunima.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **Napomena:** Možda ste primijetili parametar `use_pack_sequence` koji prosljeđujemo funkciji za treniranje. Trenutno, funkcija `pack_padded_sequence` zahtijeva da tensor duljine sekvence bude na CPU uređaju, te stoga funkcija za treniranje mora izbjeći premještanje podataka o duljini sekvence na GPU tijekom treniranja. Možete pogledati implementaciju funkcije `train_emb` u datoteci [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## Dvosmjerni i višeslojni RNN-ovi

U našim primjerima, sve rekurentne mreže radile su u jednom smjeru, od početka sekvence do kraja. To izgleda prirodno, jer podsjeća na način na koji čitamo i slušamo govor. Međutim, budući da u mnogim praktičnim slučajevima imamo nasumičan pristup ulaznoj sekvenci, moglo bi imati smisla pokrenuti rekurentne izračune u oba smjera. Takve mreže nazivaju se **dvosmjerni** RNN-ovi, a mogu se stvoriti dodavanjem parametra `bidirectional=True` konstruktoru RNN/LSTM/GRU.

Kod rada s dvosmjernom mrežom, trebala bi nam dva vektora skrivenog stanja, po jedan za svaki smjer. PyTorch kodira te vektore kao jedan vektor dvostruko veće veličine, što je prilično praktično, jer biste obično proslijedili dobiveno skriveno stanje potpuno povezanoj linearnoj sloju, i samo trebate uzeti u obzir ovo povećanje veličine prilikom stvaranja sloja.

Rekurentna mreža, bilo jednosmjerna ili dvosmjerna, hvata određene uzorke unutar sekvence i može ih pohraniti u vektor stanja ili proslijediti u izlaz. Kao i kod konvolucijskih mreža, možemo izgraditi još jedan rekurentni sloj na vrhu prvog kako bismo uhvatili uzorke višeg nivoa, izgrađene od uzoraka nižeg nivoa koje je izvukao prvi sloj. To nas dovodi do pojma **višeslojni RNN**, koji se sastoji od dvije ili više rekurentnih mreža, gdje se izlaz prethodnog sloja prosljeđuje sljedećem sloju kao ulaz.

![Slika koja prikazuje višeslojni long-short-term-memory RNN](../../../../../translated_images/hr/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Slika iz [ovog sjajnog posta](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) autora Fernanda Lópeza*

PyTorch olakšava izgradnju takvih mreža, jer samo trebate dodati parametar `num_layers` konstruktoru RNN/LSTM/GRU kako biste automatski izgradili nekoliko slojeva rekurencije. To bi također značilo da će se veličina vektora skrivenog stanja proporcionalno povećati, i trebate to uzeti u obzir prilikom rukovanja izlazom rekurentnih slojeva.


## RNN-ovi za druge zadatke

U ovoj jedinici vidjeli smo da se RNN-ovi mogu koristiti za klasifikaciju sekvenci, ali zapravo mogu obraditi mnogo više zadataka, poput generiranja teksta, strojnog prevođenja i drugih. Te zadatke ćemo razmotriti u sljedećoj jedinici.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
